In [ ]:
%pip install --upgrade --quiet pip
%pip install --upgrade --quiet watermark circuitree[distributed]==0.11.1

# ## Uncomment these lines if you are using Colab
# !git clone https://github.com/pranav-bhamidipati/circuitree-tutorial.git
# %cd /content/circuitree-tutorial/src

# Parallel MCTS with CircuiTree

MCTS is an iterative sampling algorithm, where the reward found in each iteration affects sampling in later iterations. While perfect parallel execution isn't possible, we can achieve quite good performance using the so-called lock-free method [[1]](https://doi.org/10.1007/978-3-642-12993-3_2), where multiple multiple search threads in the same CPU (the *main node*) are running MCTS concurrently, each one taking turns editing the search graph. We will implement this in detail later in the tutorial, but in brief, instead of computing the (usually expensive) reward function, each search thread on the main node sends a request to a group of worker CPUs (the *worker node*) somewhere else that will do the actual computation, and while that thread is waiting for the result, other search threads can use the main CPU. As long as our execution time is significantly longer than the time spent sending and receiving those signals, we should see a performance boost!

## Parallel CircuiTree on a single machine

In order to parallelize the search on a local machine, we can nominate a group of CPUs in our own computer to be the worker node that performs reward function evaluations. We can coordinate the main and worker nodes using a *producer-consumer* queue. The main node will produce tasks (calls to the reward function) that get added to the queue, and the worker node will consume tasks from the queue and return the result to a shared database where the main node can look up the result. We'll manage this task queue with the Python utility `celery`. 

Here's a schematic of how that infrastructure looks.

![Local-Infrastructure](./local_parallel_infrastructure.png)

### Steps to running a parallel search
1) Set up an in-memory database.
2) Package the reward function into a `celery` app.
3) Define a `CircuiTree` subclass that calls the reward function in (2).
4) Launch some workers.
5) Run the search script.

### 1. Database installation

We will be using a lightweight database called Redis (https://redis.io/). If you are running this notebook on Colab or on a machine without Redis installed, you can uncomment and run the next code block to install Redis. Otherwise, please skip the next code block and follow the installation instructions [here](https://redis.io/docs/latest/operate/oss_and_stack/install/install-redis/) instead. If you are using a Redis server hosted somewhere else, you can skip the next code block and change the `host` and `port` arguments later in the notebook to point to your server.

__NOTE:__ While you can perform all 4 setup steps in Colab, free-tier Colab notebooks only run on 1 CPU. So unless you are using an upgraded VM, you will not notice a speed improvement.

In [ ]:
####################################################################################
### If you are using Colab, uncomment and run this :) ##############################
####################################################################################
# # Download the latest stable release and make from source (can take 4+ minutes)
# !curl -o ./redis-server.tar.gz -fsSL https://download.redis.io/redis-stable.tar.gz
# !tar -xf ./redis-server.tar.gz
# !cd ./redis-stable && make # Can take a while (5+ minutes)
# !/content/redis-stable/src/redis-server --daemonize yes

**Be sure to test your installation!**

In [ ]:
## This should print "PONG"

# Colab notebook users
# !/content/redis-stable/src/redis-cli ping 

# Local installations
!redis-cli ping 

### 2. Making a `celery` app with the reward function 
The app is a Python script that tells `celery` where the database is and which tasks it will be managing. For instance, here is the script for the bistability app.

In [ ]:
from pathlib import Path

print(Path("bistability_app.py").read_text())

We use the `Celery` command to create an app that uses the `Redis` database to pass messages (the `broker` option) and store results (the `backend` argument). The URL here points to the default location for a local database (port `6379` on the `localhost` network). Any function with the `@app.task` decorator becomes a `celery` *task* that can be executed by a worker - we'll see how this looks in the next section.

### 3. Calling the reward function as a `celery` task

Unlike a normal function call, a call to a `celery` task is *asynchronous*. This means that when the main node calls the function, it dispatches a task to the workers, and the result can be requested later. This uses different syntax - instead of running `reward = get_reward(...)` directly, we run `result = get_reward_celery.delay(...)` to dispatch the task from the main node to the workers. This immediately returns an `AsyncResult` object that can be inspected to monitor progress. Then, once we need the result, we call `future.get()` and wait for the reward to arrive. While one thread is waiting for the reply, another thread can take over the main node and run a search iteration. 

All we need to do in this step is make a new subclass of `CircuiTree` that runs the reward function using the app. Here's what that looks like in our bistability example.

In [ ]:
print(Path("bistability_parallel.py").read_text())

Python's `threading` module can manage up to a few dozen threads, but we want to run a search with hundreds to thousands of threads. For this, we will use the `gevent` module, which re-defines many of the built-in Python commands in order to support its highly scalable "green threads." Re-defining built-in code is called "monkey patching," and it has to be run as the first line in the file where we define the class. (We also ran `monkey.patch_all()` at the beginning of this notebook - this is only necessary for notebooks, not for scripts.)

### 4. Launching a worker node

We can launch a worker node using `celery`'s command line interface. To do so, open a separate terminal, `cd` to the folder with the app, activate your virtual environment, and run the following command, replacing the `XX` with the number of CPUs to use. (If you aren't using a virtual environment, you should!)

```
# Launch a worker called 'worker1' with 'XX' CPUs, specifying the app with the '.app' suffix.
# If you supply the 'logfile' flag, the worker will write its logs to there
celery --app bistability_app.app multi start "worker1" --concurrency=XX --loglevel=INFO #--logfile="./worker1.log"
```

ALternatively, if you're running this in Colab, uncomment and run this cell:

In [ ]:
# # Uncomment on Colab
# !celery --app bistability_app.app worker --concurrency=XX --loglevel=INFO --detach --logfile=./worker1.log

To see how many CPUs you have on a Linux machine, run `lscpu | grep '^CPU(s)'`. It's best practice to subtract one or two CPUs from that number when launching a worker, since performance can degrade if you try to use every single CPU on the machine.

### 5. Running a parallel search

Now we can run the search by running a script from the main node. If you look at the terminal where the worker is launched, you should see the progress! Without parallel execution, this script would run 10,000 iterations in 17 minutes.

In [ ]:
!pip install --quiet upgrade pip
!pip install --quiet -r ../requirements.py

!python ./run_search_parallel.py



That's it! To analyze the results, we can read the tree object from file

In [ ]:
from circuitree import SimpleNetworkGrammar
from bistability import BistabilityTree

# Get the file paths for the data
data_dir = Path("./tree-backups")
gml_file = list(data_dir.glob("*parallel_bistability_search*.gml"))[0]
json_file = list(data_dir.glob("*parallel_bistability_search*.json"))[0]

# Read from file. Note that we need to specify the class of the grammar
tree = BistabilityTree.from_file(gml_file, json_file, grammar_cls=SimpleNetworkGrammar)

---



## Distributed tree search

The same framework we use to run the search in parallel on a local machine can be used to run a search across many machines, in the cloud! 

![Distributed-Infrastructure](./distributed_infrastructure.png)


There are a few differences. Notably, the in-memory database now lives in a remote machine. The communication between main and worker nodes can be a bottleneck in scaling up this infrastructure, so it is important that your database has fast, high-bandwidth networking and is on the same network as the main and worker nodes. Most cloud providers already have a solution for this (for example, as of May 2024, Amazon Elasticache + EC2). The main node has the search graph and makes backups, so it will generally need higher memory, while worker nodes should have higher computing resources. Also, because Celery does not make it obvious where each task is run, you should take care that your backups are being saved to the correct location on the correct machine. Celery generally provides very robust logging as well, so be sure to specify the `--logfile` option in the `celery worker` command to take advantage of it. 

---

In [ ]:
%load_ext watermark
%watermark -v -p circuitree,numpy,jupyterlab,celery,redis,gevent,watermark